<a href="https://colab.research.google.com/github/123vartika123/MedGPT-Insights/blob/main/Data_Driven_Medical_Intelligence_Scraping_and_Training_a_Private_GPT_on_Top_Hospital_Profiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [45]:
# List of Practo hospital URLs
urls = [
    "https://www.practo.com/Noida/hospital/fortis-noida-sector-62-noida/doctors",
    "https://www.practo.com/Noida/hospital/max-hospital-vaishali/doctors",
    "https://www.practo.com/Noida/hospital/jaypee-hospital-sector-128-noida/doctors",
    "https://www.practo.com/Noida/hospital/metro-hospital-sector-11-noida/doctors",
    "https://www.practo.com/Noida/hospital/yatharth-super-speciality-hospital-noida-extension/doctors",
    "https://www.practo.com/Mumbai/hospital/fortis-hospital-mumbai-hospital/doctors",
    "https://www.practo.com/Mumbai/hospital/sunrise-hospital-mumbai/doctors",
    "https://www.practo.com/Mumbai/hospital/joy-hospital-mumbai/doctors",
    "https://www.practo.com/Mumbai/hospital/kurla-hospital-mumbai/doctors",
    "https://www.practo.com/Mumbai/hospital/saifee-hospital-mumbai/doctors",
    "https://www.practo.com/Delhi/hospital/fortis-hospital-delhi/doctors",
    "https://www.practo.com/Delhi/hospital/max-hospital-delhi/doctors",
    "https://www.practo.com/Delhi/hospital/apollo-hospital-delhi/doctors",
    "https://www.practo.com/Delhi/hospital/medanta-hospital-delhi/doctors",
    "https://www.practo.com/Delhi/hospital/blk-hospital-delhi/doctors",
    "https://www.practo.com/Bangalore/hospital/fortis-hospital-bangalore/doctors",
    "https://www.practo.com/Bangalore/hospital/maheshwari-hospital-bangalore/doctors",
    "https://www.practo.com/Bangalore/hospital/sri-balaji-hospital-bangalore/doctors",
    "https://www.practo.com/Bangalore/hospital/vikram-hospital-bangalore/doctors",
    "https://www.practo.com/Bangalore/hospital/columbia-asia-hospital-bangalore/doctors",
    "https://www.practo.com/Chennai/hospital/fortis-hospital-chennai/doctors",
    "https://www.practo.com/Chennai/hospital/apollo-hospital-chennai/doctors",
    "https://www.practo.com/Chennai/hospital/chettinad-hospital-chennai/doctors",
    "https://www.practo.com/Chennai/hospital/rajiv-gandhi-hospital-chennai/doctors",
    "https://www.practo.com/Chennai/hospital/sri-ramachandra-hospital-chennai/doctors",
    "https://www.practo.com/Kolkata/hospital/fortis-hospital-kolkata/doctors",
    "https://www.practo.com/Kolkata/hospital/saroj-gupta-hospital-kolkata/doctors",
    "https://www.practo.com/Kolkata/hospital/narayana-health-kolkata/doctors",
    "https://www.practo.com/Kolkata/hospital/medica-hospital-kolkata/doctors",
    "https://www.practo.com/Kolkata/hospital/peerless-hospital-kolkata/doctors",
    "https://www.practo.com/Hyderabad/hospital/medicover-hospital-hyderabad/doctors",
    "https://www.practo.com/Hyderabad/hospital/nihon-hospital-hyderabad/doctors",
    "https://www.practo.com/Hyderabad/hospital/king-koti-hospital-hyderabad/doctors",
    "https://www.practo.com/Hyderabad/hospital/clean-slate-hospital-hyderabad/doctors",
    "https://www.practo.com/Hyderabad/hospital/max-cure-hospital-hyderabad/doctors",
    "https://www.practo.com/Pune/hospital/sassoon-hospital-pune/doctors",
    "https://www.practo.com/Pune/hospital/kolhapur-hospital-pune/doctors",
    "https://www.practo.com/Pune/hospital/balaji-hospital-pune/doctors",
    "https://www.practo.com/Pune/hospital/aarogya-hospital-pune/doctors",
    "https://www.practo.com/Pune/hospital/sanjeevani-hospital-pune/doctors",
    "https://www.practo.com/Jaipur/hospital/mahatma-gandhi-hospital-jaipur/doctors",
    "https://www.practo.com/Jaipur/hospital/fortis-hospital-jaipur/doctors",
    "https://www.practo.com/Jaipur/hospital/medanta-jaipur/doctors",
    "https://www.practo.com/Jaipur/hospital/sushrut-hospital-jaipur/doctors",
    "https://www.practo.com/Jaipur/hospital/jk-lakshmipat-hospital-jaipur/doctors",
    "https://www.practo.com/Chandigarh/hospital/pgimer-chandigarh/doctors",
    "https://www.practo.com/Chandigarh/hospital/fortis-hospital-chandigarh/doctors",
    "https://www.practo.com/Chandigarh/hospital/sukhmani-hospital-chandigarh/doctors",
    "https://www.practo.com/Chandigarh/hospital/saraswati-hospital-chandigarh/doctors",
    "https://www.practo.com/Chandigarh/hospital/max-hospital-chandigarh/doctors",
    "https://www.practo.com/Indore/hospital/choithram-hospital-indore/doctors",
    "https://www.practo.com/Indore/hospital/sanjeevani-hospital-indore/doctors",
    "https://www.practo.com/Indore/hospital/life-care-hospital-indore/doctors",
    "https://www.practo.com/Indore/hospital/surya-hospital-indore/doctors",
    "https://www.practo.com/Indore/hospital/medilife-hospital-indore/doctors",
    "https://www.practo.com/Lucknow/hospital/king-george-medical-university-lucknow/doctors",
    "https://www.practo.com/Lucknow/hospital/sushila-hospital-lucknow/doctors",
    "https://www.practo.com/Lucknow/hospital/medanta-hospital-lucknow/doctors",
    "https://www.practo.com/Lucknow/hospital/ram-manohar-lohia-lucknow/doctors",
    "https://www.practo.com/lucknow/hospital/apollomedics-super-speciality-hospital-lucknow-lda-colony-sector-g/doctors"
]


In [46]:
# Function to extract hospital name and location from URL
def extract_info_from_url(url):
    # Extract location and hospital name from URL
    location_match = re.search(r'\/([a-zA-Z-]+)\/hospital\/', url)
    hospital_name_match = re.search(r'hospital\/([^\/]+)\/doctors', url)

    location = location_match.group(1) if location_match else 'Unknown'
    hospital_name = hospital_name_match.group(1).replace('-', ' ').title() if hospital_name_match else 'Unknown'

    return hospital_name, location

# Function to scrape doctor details from Practo
def scrape_doctor_details(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract hospital name and location from the URL
        hospital_name, location = extract_info_from_url(url)

        # Find all doctor entries
        doctor_entries = soup.find_all('div', class_='o-media')

        # Extract doctor details
        doctor_details = []
        for entry in doctor_entries:
            name_element = entry.find('h2', {'data-qa-id': 'doctor_name'})
            qualification_element = entry.find('div', {'data-qa-id': 'doctor_qualification'})
            experience_element = entry.find('h3', {'data-qa-id': 'doctor_experience'})
            specialization_element = entry.find('h3', {'data-qa-id': 'doctor_specialisation'})
            photo_element = entry.find('img', {'data-qa-id': 'doctor_profile_photo'})

            # Check if elements are found before accessing
            name = name_element.text.strip() if name_element else 'N/A'
            qualification = qualification_element.text.strip() if qualification_element else 'N/A'
            experience = experience_element.text.strip() if experience_element else 'N/A'
            specialization = specialization_element.text.strip() if specialization_element else 'N/A'
            photo_url = photo_element['src'] if photo_element else 'N/A'

            doctor_details.append({
                'Doctor Name': name,
                'Qualification': qualification,
                'Experience': experience,
                'Specialization': specialization,
                'Photo URL': photo_url,
                'Hospital Name': hospital_name,
                'Location': location
            })

        return doctor_details
    except requests.RequestException as e:
        print(f"Request failed for URL : {e}")
        #print(f"Request failed for URL {url}: {e}")
        return []


In [47]:
# List to store all scraped data
all_doctor_details = []

# Loop through each URL

# Loop through each URL and scrape data
for url in urls:
    print(f"Scraping URL: {url}")
    doctor_details = scrape_doctor_details(url)
    all_doctor_details.extend(doctor_details)

Scraping URL: https://www.practo.com/Noida/hospital/fortis-noida-sector-62-noida/doctors
Scraping URL: https://www.practo.com/Noida/hospital/max-hospital-vaishali/doctors
Scraping URL: https://www.practo.com/Noida/hospital/jaypee-hospital-sector-128-noida/doctors
Request failed for URL : 404 Client Error: Not Found for url: https://www.practo.com/Noida/hospital/jaypee-hospital-sector-128-noida/doctors
Scraping URL: https://www.practo.com/Noida/hospital/metro-hospital-sector-11-noida/doctors
Request failed for URL : 404 Client Error: Not Found for url: https://www.practo.com/Noida/hospital/metro-hospital-sector-11-noida/doctors
Scraping URL: https://www.practo.com/Noida/hospital/yatharth-super-speciality-hospital-noida-extension/doctors
Request failed for URL : 404 Client Error: Not Found for url: https://www.practo.com/Noida/hospital/yatharth-super-speciality-hospital-noida-extension/doctors
Scraping URL: https://www.practo.com/Mumbai/hospital/fortis-hospital-mumbai-hospital/doctors
Re

In [48]:
# Create a DataFrame and save to CSV
df = pd.DataFrame(all_doctor_details)

df.to_csv('doctors_details_list.csv', index=False)

print("Scraping completed and data saved to doctors_details_list.csv")
df

Scraping completed and data saved to doctors_details_list.csv


,Doctor Name,Qualification,Experience,Specialization,Photo URL,Hospital Name,Location
0,Dr. Vinod Vasistha,MBBS,31 years experience overall,General Physician,N/A,Fortis Noida Sector 62 Noida,Noida
1,N/A,N/A,N/A,N/A,N/A,Fortis Noida Sector 62 Noida,Noida
2,Dr. Savyasachi Saxena,"MBBS, MS - ENT",21 years experience overall,ENT/ Otorhinolaryngologist,N/A,Fortis Noida Sector 62 Noida,Noida
3,N/A,N/A,N/A,N/A,N/A,Fortis Noida Sector 62 Noida,Noida
4,Dr. Manu Tiwari,"MBBS, DPM (Psychiatry)",14 years experience overall,Psychiatrist,N/A,Fortis Noida Sector 62 Noida,Noida
...,...,...,...,...,...,...,...
75,N/A,N/A,N/A,N/A,N/A,Apollomedics Super Speciality Hospital Lucknow...,lucknow
76,Dr. Mayank Somani,MBBS,27 years experience overall,General Physician,N/A,Apollomedics Super Speciality Hospital Lucknow...,lucknow
77,N/A,N/A,N/A,N/A,N/A,Apollomedics Super Speciality Hospital Lucknow...,lucknow
78,Dr. Om Prakash Verma,"MBBS, MD - Tuberculosis & Respiratory Diseases...",25 years experience overall,Pulmonologist,N/A,Apollomedics Super Speciality Hospital Lucknow...,lucknow


In [49]:
# import pandas as pd

# Load the CSV file
#df = pd.read_csv('doctors_details_list.csv')

# Display basic information about the DataFrame
print("Initial DataFrame shape:", df.shape)
print(df.info())

Initial DataFrame shape: (80, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doctor Name     80 non-null     object
 1   Qualification   80 non-null     object
 2   Experience      80 non-null     object
 3   Specialization  80 non-null     object
 4   Photo URL       80 non-null     object
 5   Hospital Name   80 non-null     object
 6   Location        80 non-null     object
dtypes: object(7)
memory usage: 4.5+ KB
None


In [50]:
# Remove duplicate rows
df = df.drop_duplicates()
# Display basic information about the DataFrame
print("Initial DataFrame shape:", df.shape)
print(df.info())

Initial DataFrame shape: (44, 7)
<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 0 to 78
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doctor Name     44 non-null     object
 1   Qualification   44 non-null     object
 2   Experience      44 non-null     object
 3   Specialization  44 non-null     object
 4   Photo URL       44 non-null     object
 5   Hospital Name   44 non-null     object
 6   Location        44 non-null     object
dtypes: object(7)
memory usage: 2.8+ KB
None


In [51]:
# Handle missing values
# Replace 'N/A' or empty strings with NaN
df.replace('N/A', pd.NA, inplace=True)
df.replace('', pd.NA, inplace=True)
# Display basic information about the DataFrame
print("Initial DataFrame shape:", df.shape)
print(df.info())

Initial DataFrame shape: (44, 7)
<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 0 to 78
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doctor Name     40 non-null     object
 1   Qualification   40 non-null     object
 2   Experience      40 non-null     object
 3   Specialization  39 non-null     object
 4   Photo URL       0 non-null      object
 5   Hospital Name   44 non-null     object
 6   Location        44 non-null     object
dtypes: object(7)
memory usage: 2.8+ KB
None


<ipython-input-51-f5153b1423c2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace('N/A', pd.NA, inplace=True)
<ipython-input-51-f5153b1423c2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace('', pd.NA, inplace=True)


In [52]:
# Drop the 'Photo URL' column
df = df.drop(columns=['Photo URL'])
# Display basic information about the DataFrame
print("Initial DataFrame shape:", df.shape)
print(df.info())

Initial DataFrame shape: (44, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 0 to 78
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doctor Name     40 non-null     object
 1   Qualification   40 non-null     object
 2   Experience      40 non-null     object
 3   Specialization  39 non-null     object
 4   Hospital Name   44 non-null     object
 5   Location        44 non-null     object
dtypes: object(6)
memory usage: 2.4+ KB
None


In [53]:
df

,Doctor Name,Qualification,Experience,Specialization,Hospital Name,Location
0,Dr. Vinod Vasistha,MBBS,31 years experience overall,General Physician,Fortis Noida Sector 62 Noida,Noida
1,<NA>,<NA>,<NA>,<NA>,Fortis Noida Sector 62 Noida,Noida
2,Dr. Savyasachi Saxena,"MBBS, MS - ENT",21 years experience overall,ENT/ Otorhinolaryngologist,Fortis Noida Sector 62 Noida,Noida
4,Dr. Manu Tiwari,"MBBS, DPM (Psychiatry)",14 years experience overall,Psychiatrist,Fortis Noida Sector 62 Noida,Noida
6,Dr. Piyush Varshney,"MBBS, MS - General Surgery, MCh - Urology",21 years experience overall,Urologist,Fortis Noida Sector 62 Noida,Noida
8,Dr. Ajay Agarwal,"MBBS, MD - General Medicine",33 years experience overall,Internal Medicine,Fortis Noida Sector 62 Noida,Noida
10,Dr. Aradhana Singh,"MS - Obstetrics & Gynaecology, MBBS",26 years experience overall,Gynecologist,Fortis Noida Sector 62 Noida,Noida
12,Dr. Brajesh Koushle,"MBBS, MS - Orthopaedics",27 years experience overall,Orthopedic surgeon,Fortis Noida Sector 62 Noida,Noida
14,Dr. Dushyant Nadar,"MBBS, MS - General Surgery, DNB - Urology/Geni...",23 years experience overall,Urologist,Fortis Noida Sector 62 Noida,Noida
16,Dr. Jalaj Baxi,"MBBS, MS - General Surgery, DNB - General Surgery",30 years experience overall,<NA>,Fortis Noida Sector 62 Noida,Noida


In [54]:
df = df.dropna()

# Display basic information about the DataFrame
print("Initial DataFrame shape:", df.shape)
print(df.info())

Initial DataFrame shape: (39, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 0 to 78
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Doctor Name     39 non-null     object
 1   Qualification   39 non-null     object
 2   Experience      39 non-null     object
 3   Specialization  39 non-null     object
 4   Hospital Name   39 non-null     object
 5   Location        39 non-null     object
dtypes: object(6)
memory usage: 2.1+ KB
None


In [55]:
df

,Doctor Name,Qualification,Experience,Specialization,Hospital Name,Location
0,Dr. Vinod Vasistha,MBBS,31 years experience overall,General Physician,Fortis Noida Sector 62 Noida,Noida
2,Dr. Savyasachi Saxena,"MBBS, MS - ENT",21 years experience overall,ENT/ Otorhinolaryngologist,Fortis Noida Sector 62 Noida,Noida
4,Dr. Manu Tiwari,"MBBS, DPM (Psychiatry)",14 years experience overall,Psychiatrist,Fortis Noida Sector 62 Noida,Noida
6,Dr. Piyush Varshney,"MBBS, MS - General Surgery, MCh - Urology",21 years experience overall,Urologist,Fortis Noida Sector 62 Noida,Noida
8,Dr. Ajay Agarwal,"MBBS, MD - General Medicine",33 years experience overall,Internal Medicine,Fortis Noida Sector 62 Noida,Noida
10,Dr. Aradhana Singh,"MS - Obstetrics & Gynaecology, MBBS",26 years experience overall,Gynecologist,Fortis Noida Sector 62 Noida,Noida
12,Dr. Brajesh Koushle,"MBBS, MS - Orthopaedics",27 years experience overall,Orthopedic surgeon,Fortis Noida Sector 62 Noida,Noida
14,Dr. Dushyant Nadar,"MBBS, MS - General Surgery, DNB - Urology/Geni...",23 years experience overall,Urologist,Fortis Noida Sector 62 Noida,Noida
18,Dr. J S Bhogal,"MBBS, MD - General Medicine, DNB - Gastroenter...",37 years experience overall,Gastroenterologist,Fortis Noida Sector 62 Noida,Noida
20,Dr. Gaurav Minocha,"MBBS, MD - General Medicine, DM - Cardiology",27 years experience overall,Interventional Cardiologist,Max Hospital Vaishali,Noida


In [56]:
# Fill missing values in 'Doctor Name' with 'Unknown'
#df['Doctor Name'].fillna('Unknown', inplace=True)

# Optional: Fill other columns with a default value if needed
# df['Qualification'].fillna('Not Specified', inplace=True)
# df['Experience'].fillna('Not Specified', inplace=True)
# df['Specialization'].fillna('Not Specified', inplace=True)
# df['Photo URL'].fillna('No Photo Available', inplace=True)

# Standardize text formats
df['Doctor Name'] = df['Doctor Name'].str.title()
df['Qualification'] = df['Qualification'].str.title()
df['Experience'] = df['Experience'].str.title()
df['Specialization'] = df['Specialization'].str.title()

# Remove any leading or trailing spaces in text fields
df['Doctor Name'] = df['Doctor Name'].str.strip()
df['Qualification'] = df['Qualification'].str.strip()
df['Experience'] = df['Experience'].str.strip()
df['Specialization'] = df['Specialization'].str.strip()

# Save the cleaned DataFrame to a new CSV file
df.to_csv('cleaned_doctors_details_list.csv', index=False)


<ipython-input-56-6581d904da98>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Doctor Name'] = df['Doctor Name'].str.title()
<ipython-input-56-6581d904da98>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Qualification'] = df['Qualification'].str.title()
<ipython-input-56-6581d904da98>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [57]:

print("Data cleaning completed and saved to cleaned_doctors_details_list.csv")
df


Data cleaning completed and saved to cleaned_doctors_details_list.csv


,Doctor Name,Qualification,Experience,Specialization,Hospital Name,Location
0,Dr. Vinod Vasistha,Mbbs,31 Years Experience Overall,General Physician,Fortis Noida Sector 62 Noida,Noida
2,Dr. Savyasachi Saxena,"Mbbs, Ms - Ent",21 Years Experience Overall,Ent/ Otorhinolaryngologist,Fortis Noida Sector 62 Noida,Noida
4,Dr. Manu Tiwari,"Mbbs, Dpm (Psychiatry)",14 Years Experience Overall,Psychiatrist,Fortis Noida Sector 62 Noida,Noida
6,Dr. Piyush Varshney,"Mbbs, Ms - General Surgery, Mch - Urology",21 Years Experience Overall,Urologist,Fortis Noida Sector 62 Noida,Noida
8,Dr. Ajay Agarwal,"Mbbs, Md - General Medicine",33 Years Experience Overall,Internal Medicine,Fortis Noida Sector 62 Noida,Noida
10,Dr. Aradhana Singh,"Ms - Obstetrics & Gynaecology, Mbbs",26 Years Experience Overall,Gynecologist,Fortis Noida Sector 62 Noida,Noida
12,Dr. Brajesh Koushle,"Mbbs, Ms - Orthopaedics",27 Years Experience Overall,Orthopedic Surgeon,Fortis Noida Sector 62 Noida,Noida
14,Dr. Dushyant Nadar,"Mbbs, Ms - General Surgery, Dnb - Urology/Geni...",23 Years Experience Overall,Urologist,Fortis Noida Sector 62 Noida,Noida
18,Dr. J S Bhogal,"Mbbs, Md - General Medicine, Dnb - Gastroenter...",37 Years Experience Overall,Gastroenterologist,Fortis Noida Sector 62 Noida,Noida
20,Dr. Gaurav Minocha,"Mbbs, Md - General Medicine, Dm - Cardiology",27 Years Experience Overall,Interventional Cardiologist,Max Hospital Vaishali,Noida


In [58]:
import pandas as pd
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load the data
data = pd.read_csv('cleaned_doctors_details_list.csv')

# Combine the relevant fields into a single text field for training the model
data['text'] = (
    "Doctor Name: " + data['Doctor Name'].astype(str) +
    ", Qualification: " + data['Qualification'].astype(str) +
    ", Experience: " + data['Experience'].astype(str) +
    ", Specialization: " + data['Specialization'].astype(str) +
    ", Hospital Name: " + data['Hospital Name'].astype(str) +
    ", Location: " + data['Location'].astype(str)
)

# Optional: Check for missing values and clean up if needed
data['text'].dropna(inplace=True)

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the text
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding='max_length', max_length=128)

# Apply the tokenization correctly - ensuring that the text input is a string
data['tokens'] = data['text'].apply(lambda x: tokenize_function(str(x))['input_ids'])

# Check out the tokenized data
print(data['tokens'].head())

# Convert tokens to PyTorch tensors for model training
tokens_tensor = torch.stack([torch.tensor(t) for t in data['tokens'].tolist()])
train_dataset = TensorDataset(tokens_tensor)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Now you can proceed with training your model using train_loader


0    [37564, 6530, 25, 1583, 13, 11820, 375, 23663,...
1    [37564, 6530, 25, 1583, 13, 311, 2830, 292, 14...
2    [37564, 6530, 25, 1583, 13, 1869, 84, 309, 142...
3    [37564, 6530, 25, 1583, 13, 350, 7745, 1530, 5...
4    [37564, 6530, 25, 1583, 13, 22028, 323, 2449, ...
Name: tokens, dtype: object


In [59]:
from transformers import GPT2LMHeadModel, GPT2Config, AdamW
from transformers import get_scheduler

# Load pre-trained GPT2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the number of epochs
num_epochs = 3

# Define scheduler (optional)
total_steps = len(train_loader) * num_epochs
scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [60]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch = batch[0].to(device)
        # Forward pass
        outputs = model(input_ids=batch, labels=batch)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} Loss: {epoch_loss / len(train_loader)}")


Epoch 1/3: 100%|██████████| 5/5 [01:40<00:00, 20.16s/it]


Epoch 1/3 Loss: 4.646397447586059


Epoch 2/3: 100%|██████████| 5/5 [01:24<00:00, 16.96s/it]


Epoch 2/3 Loss: 1.8784904718399047


Epoch 3/3: 100%|██████████| 5/5 [01:20<00:00, 16.00s/it]

Epoch 3/3 Loss: 1.679418110847473


In [61]:
model.save_pretrained('trained_gpt_model')
tokenizer.save_pretrained('trained_gpt_model')


('trained_gpt_model/tokenizer_config.json',
 'trained_gpt_model/special_tokens_map.json',
 'trained_gpt_model/vocab.json',
 'trained_gpt_model/merges.txt',
 'trained_gpt_model/added_tokens.json')

In [62]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_gpt_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_gpt_model')

# Set the model to evaluation mode
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)